In [1]:
import pandas as pd
df=pd.read_csv('quora_duplicate_questions.tsv',sep='\t')
df.dropna(inplace=True)
df=df.iloc[300000:,:]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104287 entries, 300002 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            104287 non-null  int64 
 1   qid1          104287 non-null  int64 
 2   qid2          104287 non-null  int64 
 3   question1     104287 non-null  object
 4   question2     104287 non-null  object
 5   is_duplicate  104287 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 5.6+ MB


In [2]:
df.drop(columns=['id','qid1','qid2',],inplace=True)

In [3]:
df=df.query('is_duplicate==0')
df['is_duplicate'].sum()

0

In [4]:
from transformers import pipeline
import tensorflow as tf
model = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')
positive_grad=['amusement','excitement','pride','optimism','gratitude','joy','admiration','love','approval','caring','desire','relief']
neutral_grad=['confusion','curiosity','realization','surprise','neutral']
negative_grad=['anger','disgust','grief','fear','sadness','nervousness','annoyance','disappointment','embarrassment','remorse','disapproval']
total_grad=positive_grad+neutral_grad+negative_grad

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

def predict_emotion(text):
    if(model(text)[0]['score']<0.5):
        return('neutral')
    else:
        return model(text)[0]['label']


In [6]:
from tqdm import tqdm

for index, row in tqdm(df.iterrows(), total=len(df), desc='Processing rows'):
    input_text = row['question1']
    target_text = row['question2']

    input_emotion = predict_emotion(input_text)
    target_emotion = predict_emotion(target_text)

    same_list_check = (
        (input_emotion in positive_grad and target_emotion in positive_grad) or
        (input_emotion in neutral_grad and target_emotion in neutral_grad) or
        (input_emotion in negative_grad and target_emotion in negative_grad)
    )

    if (input_emotion == target_emotion or input_emotion == 'neutral' or target_emotion == 'neutral' or not same_list_check):
        df.drop(index, inplace=True)
    else:
        if total_grad.index(input_emotion) > total_grad.index(target_emotion):
            df.at[index, 'question1'], df.at[index, 'question2'] = df.at[index, 'question2'], df.at[index, 'question1']
            input_emotion, target_emotion = target_emotion, input_emotion
        emotion_transition = f"{input_emotion} to {target_emotion}: "
        modified_input_text = emotion_transition + input_text
        df.at[index, 'question1'] = modified_input_text


Processing rows: 100%|██████████| 66498/66498 [2:03:58<00:00,  8.94it/s]  


In [7]:
df.drop(columns=['is_duplicate'],inplace=True)
df.to_csv('quora_new_5.csv',index=False)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, 301247 to 389030
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   question1  21 non-null     object
 1   question2  21 non-null     object
dtypes: object(2)
memory usage: 504.0+ bytes
